In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" # If you have more than one GPU, use this to select the one you want to use

from matplotlib import pyplot as plt
from PIL import Image
import albumentations as A
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys
import cv2

sys.path.insert(0, "../packages/python")
from data import utils as data_utils
from data import augmentation as data_augmentation
from models import cell_segmentation as segmentators


/home/nicolas/Documentos/UTN/INA/giar_ina_dev/.venv/lib/python3.12/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
2025-03-29 12:22:11.528844: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-29 12:22:11.534793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743261731.541698   26597 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin c

In [2]:
import tensorflow as tf
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print("cuDNN Enabled:", tf.test.is_built_with_cuda())

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
cuDNN Enabled: True


# Cell Segmentation Examples

This notebook has some examples on how the ´CellMaskGenerator´ class (and its childs) work.

### Segment Anything Model (SAM)

Download checkpoint files here: https://pypi.org/project/segment-anything-py/#model-checkpoints

This models requieres 16 GB of RAM (or VRAM) to work.


In [8]:
# Location of the model, input data and output location, modify to your structure
SAM_CHECKPOINT_PATH = "../models/SAM/sam_vit_h_4b8939.pth"
IMAGE_PATH = "../media/Onion-Cell-Merged-v6.v1i.coco/valid/"
OUTPUT = "../output/"
CSV_PATH = os.path.join(OUTPUT, "cropped_cells_onion_v3/data/valid")
CROPPED_OUTPUT = os.path.join(OUTPUT, "cropped_cells_onion_v3/media/valid")
MODELS_PATH = '../models/'
# Select the devicce: 
# "cuda" : Will use the NVIDIA GPU
# "cpu" : Will use the... CPU
DEVICE_USE = "cuda"

In [4]:
# Load Model
cmg = segmentators.SAMCellMaskGenerator(SAM_CHECKPOINT_PATH, model_type = 'vit_h', device = DEVICE_USE)

In [18]:
# Apply segmentation to the whole
data_utils.dataset_cell_segmentation(cmg, IMAGE_PATH, CSV_PATH)

100%|██████████| 129/129 [06:03<00:00,  2.81s/it]


### Crop nucleai

Using the output csv with the bbox info, the segmentations are cropped from the original image and stored in CROPPED_OUTPUT

In [9]:
if not os.path.exists(CROPPED_OUTPUT):
    os.makedirs(CROPPED_OUTPUT)

for file in tqdm(sorted(os.listdir(IMAGE_PATH))):
    image_name = os.fsdecode(file)
    image_base_name, _ = os.path.splitext(image_name)
    image = os.path.join(IMAGE_PATH, image_name)
    csv_path = os.path.join(CSV_PATH, image_base_name + '.csv')
    cmg.crop_cells(image_path=image, masks_path=csv_path, output_dir=CROPPED_OUTPUT)

100%|██████████| 129/129 [00:04<00:00, 29.23it/s]


In [10]:
import shutil

# Define the target folder and the new folder to recreate
IMAGE_PATH = '../output/cropped_cells_onion_v3/media/'
TARGET_FOLDER = '../output/dataset_test/test_roboflow/cells'  # Replace with the actual path to your target folder
NEW_FOLDER = '../output/dataset_test/test_roboflow_v3/cells'  # Replace with the desired output folder

# Create the new folder if it doesn't exist
if not os.path.exists(NEW_FOLDER):
    os.makedirs(NEW_FOLDER)

# Get the list of images in the target folder
target_images = os.listdir(TARGET_FOLDER)

# Search for each image in IMAGE_PATH and copy it to the new folder
for root, dirs, files in os.walk(IMAGE_PATH):
    for file in files:
        if file in target_images and file.lower().endswith(('.png', '.jpg', '.jpeg')):
            source_path = os.path.join(root, file)
            destination_path = os.path.join(NEW_FOLDER, file)
            shutil.copy(source_path, destination_path)

### Tests

In [5]:
# Apply segmentation to the whole
data_utils.dataset_cell_segmentation(cmg, "../media/test/", "../media/test/")

100%|██████████| 2/2 [01:21<00:00, 40.90s/it]


In [ ]:
image =  '../media/test/005_00020.jpg'
df = pd.read_csv("../media/test.csv") # sam_uploaded_out

bboxes_sam_not = [tuple(map(int, row)) for row in df[['x', 'y', 'w', 'h']].values]
img = cv2.imread(image,cv2.IMREAD_GRAYSCALE)


x, y, w, h = (2663, 144, 200, 200)
x1, y1 = int(x), int(y)
x2, y2 = int(x + w), int(y + h)
cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 0), 3)

x, y, w, h = (2617, 119, 292, 250)
x1, y1 = int(x), int(y)
x2, y2 = int(x + w), int(y + h)
cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)

plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
image_name = "005_00020.jpg"
image_path =  f'../media/uploaded/{image_name}'
image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)

df = pd.read_csv("../output/sam_uploaded_out/005_00020.csv") # sam_uploaded_out
df_bbox = df[df['image'] == image_name][['x', 'y', 'w', 'h']]      

# Iterate over the bounding boxes and crop the image
for _, row in df_bbox.iterrows():
    x1, y1, w1, h1 = row
    x, y, w, h = cmg._adjust_bbox(row['x'], row['y'], row['w'], row['h'], 200 * 200)

    cv2.rectangle(image, (x1, y1), (x1 + w1, y1 + h1), (0, 0, 0), 3)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 0), 3)

    plt.figure(figsize=(10, 6))
    plt.imshow(image)
    plt.axis('off')
    plt.show()


In [9]:
cmg.crop_cells(image_path=image_path, masks_path="../output/sam_uploaded_out/005_00020.csv", output_dir="../media/")


### Dataset Augmentation

Using albumination, the dataset formed by the cropped images in CROPPED_OUTPUT is augmented

In [ ]:
def get_file_list(path):
    """
    Gets a list of all files within a specified path, including subdirectories.

    Args:
        path (str): The path to the directory.

    Returns:
        list: A list of file paths.
    """

    file_list = []
    for root, dirs, files in os.walk(path):
        for file in files:
            file_path = os.path.join(root, file)
            file_list.append(file_path)
    return file_list

In [ ]:
def augment_image(image):
    # Define the augmentation pipeline
    transform = A.Compose([
        A.Rotate(limit=(-180, 180), p=1),
        A.HorizontalFlip(),
        A.VerticalFlip(),
        A.RandomBrightnessContrast(p=0.4),
        A.RandomGamma(p=1, gamma_limit=(60, 110)),
        A.GaussNoise(p=0.2)
    ])

    augmented_image = transform(image=image)['image']

    return augmented_image

In [ ]:
output_images = get_file_list(CROPPED_OUTPUT)

for idx, image_path in enumerate(output_images):
    print(f"Image: {idx + 1}/{len(output_images)}", end='\r')

    # Read and augment image
    image = cv2.imread(image_path)
    augmented_image = augment_image(image)

    image_name = os.path.basename(image_path)
    path = os.path.dirname(image_path)
    base_name, ext = os.path.splitext(image_name)

    # Define augmented image name
    new_name = f"{base_name}_augmented{ext}"
    counter = 1
    while os.path.exists(os.path.join(path, new_name)):
        new_name = f"{base_name}_augmented_{counter}{ext}"
        counter += 1

    # Save the augmented image
    cv2.imwrite(os.path.join(path, new_name), augmented_image)




### Apply detected bounding boxes

Looping throught each image in CROPPED_OUTPUT it matches the image with the cell id in the CSV_PATH file to find the original image from where the cell was cropped. Then with the specified model it checks whether the image is a cell or not and if it is, with the bbox data in the csv it draws all the cells their corresponding bbox the in the original image and stores it in ../detected_cells

In [ ]:
CSV_PATH = os.path.join(OUTPUT, "sam_out_onion.csv")
segmentators.CellMaskGenerator.bbox_applier(model_path=os.path.join(MODELS_PATH,'VGG19.keras'), csv_path=CSV_PATH, cells_path=CROPPED_OUTPUT, images_path=IMAGE_PATH)#, encoder_path=os.path.join(MODELS_PATH, 'encoder2.keras'))

### Compare models detection

In [ ]:
def plot_image_grid(folder_path, rows=4, cols=4):
  """
  Plots a grid of images from a given folder.

  Args:
    folder_path: Path to the folder containing images.
    rows: Number of rows in the grid.
    cols: Number of columns in the grid.
  """

  fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

  # Adjust spacing between subplots
  fig.subplots_adjust(hspace=0.01, wspace=0.1)  # Reduce spacing

  image_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(('.jpg', '.png', '.jpeg'))]

  for ax, image_path in zip(axes.flat, image_paths):
    img = plt.imread(image_path)
    ax.imshow(img)
    ax.set_title(os.path.basename(image_path))
    ax.axis('off')

  plt.show()

# Example usage:
folder_path = '../detected_cells/'
plot_image_grid(folder_path, 3, 3)

In [ ]:
PATH = '../media/Onion-Cell-Merged-v6.v1i.coco/'
dataset = 'train'
images = os.listdir(f'{PATH}/{dataset}')

for image in images:
    image_data = image.split('_')
    if image_data[0] == 'annotation':
        continue
    new_name = image_data[0] + '_' + image_data[1] + '.png'
    os.rename(f'{PATH}{dataset}/{image}', f'{PATH}{dataset}/{new_name}')

In [ ]:
import json

DATASET = 'valid'
PATH = f'../media/Onion-Cell-Merged-v6.v1i.coco/{DATASET}/'

# Load the JSON file
with open(f'{PATH}annotations_coco.json', 'r') as file:
    data = json.load(file)

# Process the 'file_name' field in each image
for image in data['images']:
    original_name = image['file_name']
    
    # Extract the first letter and the number between underscores
    parts = original_name.split('_')
    if len(parts) >= 2:
        new_name = f"{parts[0]}_{parts[1]}.png"
        image['file_name'] = new_name

# Save the modified JSON back to the file
with open(f'{PATH}annotations_coco_v2.json', 'w') as file:
    json.dump(data, file, indent=4)